In [1]:
#Question 1

import nltk
import math
folder="C:\\Users\\sriram\\Desktop\\CLA\\Ngrams\\"
file="nonsense.txt"
inputFile=open(folder+file)
text=""
textArray=[]
for row in inputFile:
    if row !="\n":
        text+=row.replace(" ","").replace("\n"," ")
    else:
        textArray.append("<s> " + text.replace("\n","")+ " </s>")
        text=""

inputFile.close()

bigramsArray=[]
corpus=""
for sentence in textArray:
    tokens = sentence.lower().split()
    corpus= corpus+sentence + " "
    bigrams = (tuple(nltk.bigrams(tokens)))
    #print (bigrams)
    for bigram in bigrams:
        bigramsArray.append(bigram)

SentenceA="<s> I do not like them in a mouse. </s>".lower()
SentenceB="<s> I am Sam I am Sam </s>".lower()
SentenceC="<s> I do like them anywhere </s>".lower()
SentenceD="<s> I would like green ham and beef. </s>".lower()

corpus += SentenceA + " " + SentenceB + " " + SentenceC + " " + SentenceD
vocabulary=set(corpus.split())
print (vocabulary)

cfd = nltk.ConditionalFreqDist(bigramsArray)
cpd = nltk.ConditionalProbDist(cfd, nltk.MLEProbDist)

#Bigram probabilities
bigramsProbDict={}
for (a,b) in bigramsArray:
    if (a,b) not in bigramsProbDict:
        bigramsProbDict[(a,b)]= 1.0* cfd[a][b]/cfd[a].N()
print ("\nBigrams Probabilities\n")
print (bigramsProbDict)

out=open(folder+"bigramProb.csv","w")
for key in bigramsProbDict:
    out.write(str(key)+","+str(bigramsProbDict[key])+"\n")
    
out.close()
#Add One Smoothing

bigramsProbDictOne={}
for (a,b) in bigramsArray:
    #To get the unique bigrams
    if (a,b) not in bigramsProbDictOne:
        bigramsProbDictOne[(a,b)]=(1+cfd[a][b]) / (len(vocabulary)+cfd[a].N())
print ("\nBigrams Probabilities with add one smoothing\n")
print (bigramsProbDictOne)

out=open(folder+"bigramProbSmoothed.csv","w")
for key in bigramsProbDictOne:
    out.write(str(key)+","+str(bigramsProbDictOne[key])+"\n")
    
out.close()

#Probability of sentences
print ("\n\n Sentence Probabilities:\n")
sentences=[SentenceA,SentenceB,SentenceC,SentenceD]
for sentence in sentences:
    sentenceBigrams = tuple(nltk.bigrams(sentence.split()))
    sentProb=1
    bigramProb=[]
    for (a,b) in sentenceBigrams:
        prob=(1+cfd[a][b]) / (len(vocabulary)+cfd[a].N())
        sentProb*= prob
        bigramProb.append(prob)
    print (sentence, ":" , sentProb)
    #print (bigramProb)

{'am', 'sam', '<s>', 'with', 'a', 'them', 'there', 'would', 'and', 'not', '.', 'that', 'beef.', 'i', 'here', 'anywhere', 'Sam-I-am', '!', 'like', 'or', ',', '</s>', 'do', 'Here', 'house', 'green', 'mouse', 'Sam', 'eggs', 'you', 'in', 'I', 'Would', 'Do', '?', 'That', 'mouse.', 'ham'}

Bigrams Probabilities

{('anywhere', '.'): 1.0, ('a', 'mouse'): 0.5, ('sam-i-am', '.'): 0.3333333333333333, ('.', 'i'): 0.36363636363636365, ('eggs', 'and'): 1.0, ('there', '.'): 0.6666666666666666, ('with', 'a'): 1.0, ('not', 'like'): 1.0, ('mouse', '.'): 0.5, ('a', 'house'): 0.5, ('here', 'or'): 1.0, ('like', 'that'): 0.058823529411764705, ('or', 'there'): 1.0, ('do', 'not'): 0.9166666666666666, ('would', 'not'): 0.4, ('am', 'sam'): 0.6666666666666666, ('would', 'you'): 0.6, ('do', 'you'): 0.08333333333333333, ('<s>', 'i'): 0.7333333333333333, ('i', 'do'): 0.6875, ('ham', '.'): 0.75, ('green', 'eggs'): 1.0, ('sam-i-am', 'that'): 0.16666666666666666, ('there', '?'): 0.3333333333333333, ('like', 'them'): 0

In [6]:
textArray

['<s> I am Sam  </s>',
 '<s> I am Sam Sam I am  </s>',
 '<s> That Sam-I-am That Sam-I-am !  </s>',
 '<s> I do not like that Sam-I-am  </s>',
 '<s> Do you like green eggs and ham  </s>',
 '<s> I do not like them , Sam-I-am .  </s>',
 '<s> I do not like green eggs and ham .  </s>',
 '<s> Would you like them Here or there ?  </s>',
 '<s> I would not like them here or there .  I would not like them anywhere .  </s>',
 '<s> I do not like green eggs and ham .  I do not like them , Sam-I-am  </s>',
 '<s> Would you like them in a house ?  Would you like them with a mouse ?  </s>',
 '<s> I do not like them in a house .  I do not like them with a mouse . I do not like them here or there .  </s>',
 '<s> I do not like them anywhere .  </s>',
 '<s> I do not like green eggs and ham .  </s>',
 '<s> I do not like them , Sam-I-am .  </s>']

In [2]:
#Question 2

from random import randrange, uniform
def generate_next(word):
    #Initializing the word and the corresponding probability array for the bigrams
    #that has the "word" as the first word in the bigram
    wordArray=[]
    CumProbArray=[]
    #Initializing the cumulative probability
    probSum=0
    for (a,b) in bigramsProbDict: 
        if a==word:
            wordArray.append(b)
            probSum+=bigramsProbDict[(a,b)]
            CumProbArray.append(probSum)
            #print ((a,b),bigramsProbDict[(a,b)])

    #Generating a continuous random number between 0 and 1
    frand = uniform(0,1)
    #Choosing the first word from the word Array whose Cum. Prob is lesser than frand
    for i in range(len(CumProbArray)):
        if frand<CumProbArray[i]:
            #print (wordArray[i])
            return wordArray[i]

def generate_text(cpd, initialword):
    word = initialword
    sentence=""
    #Loops till the end of the sentence marker is seen
    while word !="</s>":
        sentence+=word+" "
        #print (word)
        #word = cpd[word].generate()
        word=generate_next(word)
    return sentence.replace("<s> ","")

'would you like them , sam-i-am that sam-i-am '

In [232]:
for i in range(10):
    print (generate_text(cpd,"<s>"))

would not like green eggs and ham . 
i do not like them anywhere . 
i do not like green eggs and ham 
i do not like them here or there . i do not like them anywhere . 
i am sam sam i do not like them in a mouse ? would you like green eggs and ham . 
i am 
i do not like them with a mouse . 
i am sam i would you like that sam-i-am 
i would not like them in a house ? would not like them here or there . 
i do not like them in a mouse ? 


In [233]:
#Question 3


inputFile=open(folder+file)
text=""
textArray=[]
for row in inputFile:
    if row !="\n":
        text+=row.replace(" ","").replace("\n"," ")
    else:
        textArray.append("**" + text.replace("\n","")+ "**")
        text=""

inputFile.close()

trigramsArray=[]
corpusChar=""
from nltk.util import ngrams
for sentence in textArray:
    corpusChar= corpusChar+sentence + " "
    trigrams = ngrams(sentence.lower(),3)
    for trigram in trigrams:
        trigramsArray.append(trigram)

SentenceA="**I do not like them in a mouse.**".lower()
SentenceB="**I am Sam I am Sam**".lower()
SentenceC="**I do like them anywhere**".lower()
SentenceD="**I would like green ham and beef.**".lower()

corpusChar += SentenceA + " " + SentenceB + " " + SentenceC + " " + SentenceD
charVocabulary=set(corpusChar)
#print (charVocabulary)
#kneser_ney = nltk.KneserNeyProbDist(cfdChar)

biCharArray=[]
corpusChar=""
from nltk.util import ngrams
for sentence in textArray:
    corpusChar= corpusChar+sentence + " "
    bigrams = ngrams(sentence.lower(),2)
    for bigram in bigrams:
        biCharArray.append(bigram)
#print (biCharArray)

#print(trigramsArray)
trigram_cnt = {}
bigram_cnt = {}

for (a,b,c) in trigramsArray:
    if (a,b,c) not in trigram_cnt:
        trigram_cnt[(a,b,c)] = 1
    else:
        trigram_cnt[(a,b,c)] += 1
#print(trigram_cnt)

for (a,b) in biCharArray:
    if (a,b) not in bigram_cnt:
        bigram_cnt[(a,b)] = 1
    else:
        bigram_cnt[(a,b)] += 1

# Calculating character trigram probabilities
trigram_probDic = {}
for (a,b,c) in trigram_cnt:
    if (a,b,c) not in trigram_probDic:
        trigram_probDic[(a,b,c)] = trigram_cnt[(a,b,c)]/ bigram_cnt[(a,b)]
print(trigram_probDic)

out=open(folder+"TrigramProb.csv","w")
for key in trigram_probDic:
    out.write(str(key)+","+str(trigram_probDic[key])+"\n")
    
out.close()

#Add One Smoothing

trigram_probDicOne={}
for (a,b,c) in trigramsArray:
    #To get the unique trigrams
    if (a,b,c) not in trigram_probDicOne:
        trigram_probDicOne[(a,b,c)]=  (trigram_cnt[(a,b,c)] + 1) / (bigram_cnt[(a,b)]+ len(charVocabulary))
print ("\nTrigrams Probabilities with add one smoothing\n")
print (trigram_probDicOne)

out=open(folder+"TrigramProbSmoothed.csv","w")
for key in trigram_probDicOne:
    out.write(str(key)+","+str(trigram_probDicOne[key])+"\n")
    
out.close()

sentences=[SentenceA,SentenceB,SentenceC,SentenceD]
#Probability of sentences
print ("\n\n Sentence Probabilities:\n")
for sentence in sentences:
    sentProb=1
    sentenceTrigrams = ngrams(sentence.lower(),3)
    print("\n sentence trigrams")
    trigramProb=[]
    for (a,b,c) in sentenceTrigrams:
        try:
            check1=trigram_cnt[(a,b,c)]
        except:
            check1=0
        try:
            check2=bigram_cnt[(a,b)] 
        except:
            check2=0
        prob=(1+check1) / (check2+ len(charVocabulary))
        sentProb*= prob
        trigramProb.append(prob)
    print (sentenceTrigrams, ":" , sentProb)
    print (trigramProb)

{('e', ' ', 'g'): 0.13793103448275862, ('n', 'y', 'w'): 1.0, (' ', 'w', 'i'): 0.4, ('h', 'o', 'u'): 1.0, (' ', 'd', 'o'): 1.0, ('y', 'o', 'u'): 1.0, (' ', 't', 'h'): 1.0, ('r', 'e', 'e'): 0.3333333333333333, ('o', ' ', 'y'): 0.08333333333333333, ('a', 'n', 'd'): 0.6666666666666666, ('m', ' ', 's'): 0.10714285714285714, ('a', 't', ' '): 1.0, (' ', 'a', 'm'): 0.23076923076923078, ('g', 's', ' '): 1.0, ('*', '*', 't'): 0.03333333333333333, ('e', ' ', 'o'): 0.10344827586206896, (' ', ' ', 'w'): 0.25, ('e', 'e', 'n'): 1.0, ('e', 'r', 'e'): 1.0, ('h', 'a', 't'): 0.42857142857142855, (' ', '!', ' '): 1.0, ('s', 'a', 'm'): 1.0, (' ', ' ', 'i'): 0.75, ('d', ' ', 'y'): 0.3333333333333333, ('*', 'd', 'o'): 1.0, ('a', 'n', 'y'): 0.3333333333333333, ('?', ' ', '*'): 0.6666666666666666, ('!', ' ', '*'): 1.0, ('d', ' ', 'h'): 0.4444444444444444, ('h', 'a', 'm'): 0.5714285714285714, (' ', 's', 'a'): 1.0, ('a', ' ', 'h'): 0.5, ('o', ' ', 'n'): 0.9166666666666666, ('o', 'u', 'l'): 0.38461538461538464, (